# Kale <3 Parsl

<img src=https://www.fodmapeveryday.com/wp-content/uploads/2017/12/fennel-kale-and-parsley-for-salad.jpg width=500px/>
<br />

Here, we demonstrate using Kale to visualize and delay execution of Parsl workflows.

### Two-way communication
- Kale intercepts Parsl execution by overriding the Parsl DataFlowKernel
- Kale executes tasks via Parsl

### Scope
The following is a brief demonstration of this interaction. So far, only Python functions have been implemented. I haven't yet tried to integrate bash functions or batch tasks as we have in Kale.

### Breaking changes
Also, I've made some changes to achieve this which have probably broken the previous Fireworks model, so some modifications will be necessary before merging to master.

In [29]:
from parsl import DataFlowKernel, ThreadPoolExecutor

from kale.workflow_widgets import WorkflowWidget, WorkerPoolWidget
from kale.parsl_dflow import KaleDFK

# Initialization

The `WorkerPoolWidget` defines how Kale tasks will be executed. The interface is definitely in progress. For example, the `Location` field doesn't do anything yet, but could be used to specify that a Workflow should execute on remote resources.

`Workers` is passed to `max_workers` in the internal ThreadPoolExecutor which will Parsl will ultimately use to execute the tasks.

In [30]:
wpw = WorkerPoolWidget()
wpw

WorkerPoolWidget(children=(HTML(value='<h3>Worker Pools</h3>'), TableWidget(children=(RowWidget(children=(HBox(children=(HTML(value='<b>Name</b>'),), layout=Layout(justify_content='flex-start', width='150px')), HBox(children=(HTML(value='<b>Location</b>'),), layout=Layout(justify_content='flex-start', width='200px')), HBox(children=(HTML(value='<b>Executor</b>'),), layout=Layout(justify_content='flex-start', width='200px')), HBox(children=(HTML(value='<b>Workers</b>'),), layout=Layout(justify_content='flex-start', width='60px')), HBox(children=(HTML(value='<b>Action</b>'),), layout=Layout(justify_content='flex-start', width='100px'))), layout=Layout(justify_content='space-between', width='710px')), RowWidget(children=(HBox(children=(HTML(value='default'),), layout=Layout(justify_content='flex-start', width='150px')), HBox(children=(HTML(value='localhost'),), layout=Layout(justify_content='flex-start', width='200px')), HBox(children=(HTML(value='parsl'),), layout=Layout(justify_content='flex-start', width='200px')), HBox(children=(HTML(value='4'),), layout=Layout(justify_content='flex-start', width='60px')), HBox(children=(Button(button_style='danger', description='Remove', style=ButtonStyle()),), layout=Layout(justify_content='flex-start', width='100px'))), layout=Layout(justify_content='space-between', width='710px')), RowWidget(children=(HBox(children=(Text(value='', layout=Layout(width='148px')),), layout=Layout(justify_content='flex-start', width='150px')), HBox(children=(Dropdown(layout=Layout(width='198px'), options=('localhost',), value='localhost'),), layout=Layout(justify_content='flex-start', width='200px')), HBox(children=(Dropdown(layout=Layout(width='198px'), options=('parsl', 'fireworks'), value='parsl'),), layout=Layout(justify_content='flex-start', width='200px')), HBox(children=(IntText(value=1, layout=Layout(width='198px')),), layout=Layout(justify_content='flex-start', width='60px')), HBox(children=(Button(button_style='success', icon='plus', style=ButtonStyle()),), layout=Layout(justify_content='flex-start', width='100px'))), layout=Layout(justify_content='space-between', width='710px'))), layout=Layout(width='710px')), HTML(value='\n        <div class="alert alert-success" style="width: 710px">\n        WorkerPool \'default\' created.\n        </div>\n        ')))

Initialize the Kale `DataFlowKernel`. I don't think this `ThreadPoolExecutor` is actually used. This interface can be cleaned up.

In [31]:
workers = ThreadPoolExecutor()
kale_dfk = KaleDFK(executors=[workers])

# Define Parsl function

Define a Parsl function as normal, except the `KaleDFK` is passed in place of the standard Parsl DFK.

In [28]:
@App('python', kale_dfk)
def rand_add(*prev_list):
    """Add a random number to the previous ones."""
    import random
    import time
    
    # Random int between 0 & 10, inclusive.
    myrand = random.randint([0, 10])
    mysum = myrand + sum(prev_list)
    
    print("My number is {}. I was given {}. The sum is {}.\n".format(myrand, prev_list, mysum), end='')
    time.sleep(2)
    
    return mysum


# Define workflow
Secify the name of the new workflow. By requiring that a Workflow be initialized, multiple workflows can be defined sequentially with the same `KaleDFK` object by calling this function in between workflow definitions.

In [ ]:
kale_dfk.new_workflow('Random Tree')

Now, instead of executing the tasks, the `KaleDFK` intercepts them and combines them into a kale `Workflow`.

In [26]:
rand_add(
    rand_add(
        rand_add(),
        rand_add()
    ),
    rand_add(
        rand_add(),
        rand_add()
    )
)

<AppFuture at 0x7fb34c62f240 state=finished returned NoneType>

# Launch workflow
A workflow widget is used to visualize and interact with the workflow once it has been defined.

Tasks can be selected in the plot using the mouse. `ctrl-click` for multi-select.

### `Workflow`
The **Run Workflow** button in the `Workflow` tab start execution. 
If some tasks are selected, only those tasks are run. Currently, an error will be raised if there's an unselected task in the middle of a dependenct chain.

### `Task`
In the `Task` tab, some basic information about the selected task is displayed, including the function name and its arguments.

### `Tags`
In the `Tags` tab, various selection utilites are available. Also, we have the concept of assigning tags to Kale tasks for easier selection and grouping. I haven't implemented this for workflows generated from Parsl.

### `Widget Log`
The `Widget Log` tab is generally where output from the functions is displayed, but I currently have it disabled, so all output will be printed below the widget.

In [27]:
ww = WorkflowWidget(kale_dfk.kale_workflow, wpw)
ww

WorkflowWidget(children=(VBox(children=(Figure(fig_margin={'left': 10, 'right': 10, 'bottom': 10, 'top': 10}, layout=Layout(height='600px', width='400px'), marks=[Graph(colors=['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf'], highlight_links=False, interactions={'hover': 'tooltip', 'click': 'select'}, link_data=[{'source': 0, 'target': 2}, {'source': 1, 'target': 2}, {'source': 2, 'target': 6}, {'source': 3, 'target': 5}, {'source': 4, 'target': 5}, {'source': 5, 'target': 6}], link_type='line', node_data=[{'label': '0', 'shape': 'rect', 'name': 'rand_add', 'task_type': 'PythonFunctionTask', 'tags': '[]', 'input_files': '[]', 'output_files': '[]', 'num_cores': '1', 'args': '[]', 'kwargs': '{}'}, {'label': '1', 'shape': 'rect', 'name': 'rand_add', 'task_type': 'PythonFunctionTask', 'tags': '[]', 'input_files': '[]', 'output_files': '[]', 'num_cores': '1', 'args': '[]', 'kwargs': '{}'}, {'label': '2', 'shape': 'rect', 'name': 'rand_add', 'task_type': 'PythonFunctionTask', 'tags': '[]', 'input_files': '[]', 'output_files': '[]', 'num_cores': '1', 'args': '[<kale.workflow_objects.PythonFunctionTask object at 0x7fb34c5b55f8>, <kale.workflow_objects.PythonFunctionTask object at 0x7fb34c5b56d8>]', 'kwargs': '{}'}, {'label': '3', 'shape': 'rect', 'name': 'rand_add', 'task_type': 'PythonFunctionTask', 'tags': '[]', 'input_files': '[]', 'output_files': '[]', 'num_cores': '1', 'args': '[]', 'kwargs': '{}'}, {'label': '4', 'shape': 'rect', 'name': 'rand_add', 'task_type': 'PythonFunctionTask', 'tags': '[]', 'input_files': '[]', 'output_files': '[]', 'num_cores': '1', 'args': '[]', 'kwargs': '{}'}, {'label': '5', 'shape': 'rect', 'name': 'rand_add', 'task_type': 'PythonFunctionTask', 'tags': '[]', 'input_files': '[]', 'output_files': '[]', 'num_cores': '1', 'args': '[<kale.workflow_objects.PythonFunctionTask object at 0x7fb34c62f278>, <kale.workflow_objects.PythonFunctionTask object at 0x7fb34c62f470>]', 'kwargs': '{}'}, {'label': '6', 'shape': 'rect', 'name': 'rand_add', 'task_type': 'PythonFunctionTask', 'tags': '[]', 'input_files': '[]', 'output_files': '[]', 'num_cores': '1', 'args': '[<kale.workflow_objects.PythonFunctionTask object at 0x7fb34c6007b8>, <kale.workflow_objects.PythonFunctionTask object at 0x7fb34c62f390>]', 'kwargs': '{}'}], scales={'x': LinearScale(), 'y': LinearScale()}, scales_metadata={'x': {'orientation': 'horizontal', 'dimension': 'x'}, 'y': {'orientation': 'vertical', 'dimension': 'y'}, 'color': {'dimension': 'color'}, 'link_color': {'dimension': 'link_color'}}, selected_style={'stroke': 'red'}, tooltip_style={'opacity': 0.9}, x=array([  335.37,  1026.4 ,  1026.4 ,  1717.4 ,  2406.4 ,  1717.4 ,  1371.4 ]), y=array([ 162.,  162.,   90.,  162.,  162.,   90.,   18.]))], min_aspect_ratio=0.0, scale_x=LinearScale(allow_padding=False, max=1.0, min=0.0), scale_y=LinearScale(allow_padding=False, max=1.0, min=0.0)), Toolbar(figure=Figure(fig_margin={'left': 10, 'right': 10, 'bottom': 10, 'top': 10}, layout=Layout(height='600px', width='400px'), marks=[Graph(colors=['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf'], highlight_links=False, interactions={'hover': 'tooltip', 'click': 'select'}, link_data=[{'source': 0, 'target': 2}, {'source': 1, 'target': 2}, {'source': 2, 'target': 6}, {'source': 3, 'target': 5}, {'source': 4, 'target': 5}, {'source': 5, 'target': 6}], link_type='line', node_data=[{'label': '0', 'shape': 'rect', 'name': 'rand_add', 'task_type': 'PythonFunctionTask', 'tags': '[]', 'input_files': '[]', 'output_files': '[]', 'num_cores': '1', 'args': '[]', 'kwargs': '{}'}, {'label': '1', 'shape': 'rect', 'name': 'rand_add', 'task_type': 'PythonFunctionTask', 'tags': '[]', 'input_files': '[]', 'output_files': '[]', 'num_cores': '1', 'args': '[]', 'kwargs': '{}'}, {'label': '2', 'shape': 'rect', 'name': 'rand_add', 'task_type': 'PythonFunctionTask', 'tags': '[]', 'input_files': '[]', 'output_

My number is 8. I was given (). The sum is 8.
My number is 10. I was given (). The sum is 10.
My number is 9. I was given (). The sum is 9.
My number is 1. I was given (). The sum is 1.
My number is 2. I was given (1, 8). The sum is 11.
My number is 5. I was given (10, 9). The sum is 24.
My number is 10. I was given (24, 11). The sum is 45.


# Future Improvements
- Only Python tasks are currently supported with Parsl. This should be expanded to bash and batch tasks.
- In the spirit of Parsl's on-the-fly workflow definition, it should be possible to dynamically update the widget if new tasks are added to the workflow. This isn't yet implemented.
- We're in the process of developing a Kale service which will facilitate monitoring and controlling the status of workflows and tasks between multiple notebooks and across resources.
- General UI improvements are underway. Suggestions are welcome!